In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\asus\.conda\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY = "ee7860c7-aab7-4bec-bd66-0521c66fe048"

In [3]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [4]:
extracted_data = load_pdf("data/")

In [5]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [6]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [7]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [8]:
embeddings = download_hugging_face_embeddings()

In [9]:
vector=[embeddings.embed_query(t.page_content) for t in text_chunks]

In [10]:
vectors=[]
for i,vec in enumerate(vector):
    vectors.append({'id':str(i),'values':vec,"metadata": {'text':text_chunks[i].page_content}})

In [12]:

import itertools
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY, pool_threads=30)
index = pc.Index("medic-bot")
def chunks(iterable, batch_size=100):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))
with pc.Index('medic-bot', pool_threads=30) as index:

    async_results = [
        index.upsert(vectors=ids_vectors_chunk, async_req=True)
        for ids_vectors_chunk in chunks(vectors, batch_size=100)
    ]
    [async_result.get() for async_result in async_results]

In [13]:
text='What are Allergies'

In [14]:
def find_similarity(text):
    query=embeddings.embed_query(text)
    response=index.query(vector=query,top_k=5,include_values=True,include_metadata=True)
    results=response.matches
    matched_data=[]
    for result in results:
        text=result.metadata['text']
        matched_data.append(text)
    return matched_data

In [15]:
find_similarity(text)

["GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE",
 'allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly cause aller-\ngic reactions:\n• penicillin or other a

In [17]:
text="what is acne disease"

In [ ]:
! pip install openai==0.28

   ---------------------------------------- 0.0/76.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/76.5 kB ? eta -:--:--
   ---------------- ----------------------- 30.7/76.5 kB 660.6 kB/s eta 0:00:01
   ------------------------------------- -- 71.7/76.5 kB 787.7 kB/s eta 0:00:01
   ---------------------------------------- 76.5/76.5 kB 713.0 kB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.24.0
    Uninstalling openai-1.24.0:
      Successfully uninstalled openai-1.24.0


In [18]:
import openai
openai.api_key = 'sk-proj-UPUTw85MiYiBtYvOIXwUT3BlbkFJLrk4Y6aNyd39U6udFgMG'

In [19]:
limit = 3750

import time
conversation_history = []
def retrieve(query):
    vector=embeddings.embed_query(query)
    # get relevant contexts
    contexts = []
    for message in conversation_history:
        contexts.append(f"{message['role'].capitalize()}: {message['content']}\n")
    time_waited = 0
    while (len(contexts) < 3 and time_waited < 60 * 12):
        res=index.query(vector=vector,top_k=3,include_values=True,include_metadata=True)
        contexts = contexts + [
            x['metadata']['text'] for x in res['matches']
        ]
        print(f"Retrieved {len(contexts)} contexts, sleeping for 15 seconds...")
        time.sleep(15)
        time_waited += 15

    if time_waited >= 60 * 12:
        print("Timed out waiting for contexts to be retrieved.")
        contexts = ["No contexts retrieved. Try to answer the question yourself!"]


    # build our prompt with the retrieved contexts included
    prompt_start = (
        "Use the following pieces of information to answer the user's question.\
        If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n"+
        "Context:\n"
    )
    prompt_end = (
        f"\n\nQuestion: {query}\n Only return the helpful answer below and nothing else.\nAnswer:"
    )
    # append contexts until hitting limit
    for i in range(1, len(contexts)):
        if len("\n\n---\n\n".join(contexts[:i])) >= limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts) +
                prompt_end
            )
    conversation_history.append({"role": "user", "content": query})
    return prompt


def complete(prompt):
    # instructions
    sys_prompt = "You are a helpful assistant that always answers questions."
    # query text-davinci-003
    res = openai.ChatCompletion.create(
        model='gpt-3.5-turbo-0613',
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    response=res['choices'][0]['message']['content'].strip()
    conversation_history.append({"role": "assistant", "content": response})
    return response

In [20]:
query_with_contexts = retrieve(text)
query_with_contexts

Retrieved 3 contexts, sleeping for 15 seconds...


"Use the following pieces of information to answer the user's question.        If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nContext:\nAcidosis seeRespiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is\n\n---\n\nGALE ENCYCLOPEDIA OF MEDICINE 2 25Acne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceousglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25\n\n---\n\nthe most common skin disease. It affects nearly 17 millionpeople in the United States. While acne can arise at anyage, it usually begins at 

In [21]:
complete(query_with_contexts)

'Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.'

In [22]:
def chatbot(query):
    query_with_contexts = retrieve(query)
    query_with_contexts
    return complete(query_with_contexts)

In [23]:
chatbot('what is acne disease')

Retrieved 5 contexts, sleeping for 15 seconds...


'Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.'

In [24]:
chatbot('what is precaution of that')

'Some precautions for acne include keeping the skin clean by washing it twice a day with a gentle cleanser, avoiding touching or picking at the pimples, using non-comedogenic skincare products, and avoiding excessive sun exposure. It is also important to maintain a healthy diet and manage stress levels, as these factors can contribute to acne.'

In [25]:
chatbot('can you summerize it?')

'Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria. Precautions for acne include keeping the skin clean, avoiding touching or picking at the pimples, using non-comedogenic skincare products, and maintaining a healthy diet and managing stress levels.'

In [26]:
chatbot('conclude whats ever I have ask till now?')

'Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria. Precautions for acne include keeping the skin clean, avoiding touching or picking at the pimples, using non-comedogenic skincare products, and maintaining a healthy diet and managing stress levels.'